## 忍者大冒险
经过前面 4 个游戏项目的制作，相信你对游戏设计已经有了基本的认识。忍者大冒险是游戏主题的最后的一个项目，难度会比前面的项目稍高。不过，复杂的项目都可以通过将项目分解来一步一步制作。先看一下这个游戏长什么样吧。

<img src='image/ch16.1.png' width=800 align=left>

游戏主角是一个会发射飞镖的忍者，他要躲避不断飞来的火球和怪物，他可以用飞镖将怪物杀死。游戏界面左上角显示了角色的状态：生命值和魔法值。魔法值充满的时候可以通过按键瞬移。游戏的挑战是尽可能得到更多的分数。介绍一下每个角色的动作：

- 忍者：左右移动和跳跃，发射飞镖，如果碰到火球或怪物生命值 -1；
- 怪物：从右向左移动，有一个怪物还能发射火球；
- 飞镖：被忍者发射后向右移动，击中怪物则得分 +1。

游戏机制分析清楚后，现在开始制作游戏吧。

### 1. 忍者
先写出游戏程序的基本框架：

In [ ]:
import pyxel


class App:
    def __init__(self):
        pyxel.init(250, 150, caption='Ninja Adventure')
        self.game_state = 'start'
        pyxel.run(self.update, self.draw)

    def update(self):
        if self.game_state == 'start':
            if pyxel.btnp(pyxel.KEY_SPACE):  
                self.game_state = 'playing'
        elif self.game_state == 'playing':
            pass

    def draw(self):
        if self.game_state == 'start':
            pyxel.text(100, 55, 'Ninja Adventure', 3)
            pyxel.text(90, 70, 'Press Space to Play!', 2)
        elif self.game_state == 'playing':
            pass

App()

导入忍者角色，并且通过按键让忍者左右移动和跳跃。游戏角色素材已经提供，读者可从 Github 自行下载：<font color=red>后续提供github 下载链接</font>

<img src='image/ch16.2.png' width=400 align=left>

In [ ]:
import pyxel


class App:
    def __init__(self):
        pyxel.init(250, 150, caption='Ninja Adventure')
        self.game_state = 'start'
        pyxel.load('ninja.pyxres')
        self.score = 0
        self.ninja_pos = [40, 112]
        self.jump = False
        self.jump_speed = 8        
        pyxel.run(self.update, self.draw)

    def update(self):
        if self.game_state == 'start':
            if pyxel.btnp(pyxel.KEY_SPACE):  
                self.game_state = 'playing'
        elif self.game_state == 'playing':
            # 忍者移动与跳跃
            if pyxel.btn(pyxel.KEY_LEFT):
                self.ninja_pos[0] -= 2
            elif pyxel.btn(pyxel.KEY_RIGHT):
                self.ninja_pos[0] += 2
            if self.ninja_pos[1] > 60:          
                if pyxel.btnp(pyxel.KEY_UP):
                    self.jump = True
                    self.jump_speed = 8
                    pyxel.play(ch=0, snd=0)
            if self.jump:
                self.ninja_pos[1] -= self.jump_speed
                self.jump_speed -= 0.6
                if self.ninja_pos[1] > 112:
                    self.ninja_pos[1] = 112
                    self.jump = False

    def draw(self):
        if self.game_state == 'start':
            pyxel.text(100, 55, 'Ninja Adventure', 3)
            pyxel.text(90, 70, 'Press Space to Play!', 2)
        elif self.game_state == 'playing':
            pyxel.cls(0)
            pyxel.text(200, 15, 'Score: ' + str(self.score), 2)
            pyxel.blt(0, 136, 1, 0, 0, 250, 2, 0)  # 画地面               
            # 画忍者
            pyxel.blt(
                self.ninja_pos[0],
                self.ninja_pos[1],
                0, 25, 0, 22, 24, 0)    

App()

运行程序，如果程序正常，应该可以通过上左右键控制忍者跳跃和移动。

<img src='image/ch16.3.png' width=800 align=left>

添加忍者发射飞镖的功能：按 S 键发射飞镖，发射飞镖时，先将飞镖移到忍者附近的位置，然后再向右移动，直到飞镖移出屏幕。

In [ ]:
import pyxel


class App:
    def __init__(self):
        pyxel.init(250, 150, caption='Ninja Adventure')
        self.game_state = 'start'
        pyxel.load('ninja.pyxres')
        self.score = 0
        self.ninja_pos = [40, 112]
        self.jump = False
        self.jump_speed = 8
        self.dart_pos = [-20, 60]  # 设置飞镖初始位置
        self.shoot = False  # 是否发射飞镖        
        pyxel.run(self.update, self.draw)

    def update(self):
        if self.game_state == 'start':
            if pyxel.btnp(pyxel.KEY_SPACE):  
                self.game_state = 'playing'
        elif self.game_state == 'playing':
            if pyxel.btn(pyxel.KEY_LEFT):
                self.ninja_pos[0] -= 2
            elif pyxel.btn(pyxel.KEY_RIGHT):
                self.ninja_pos[0] += 2
            if self.ninja_pos[1] > 60:          
                if pyxel.btnp(pyxel.KEY_UP):
                    self.jump = True
                    self.jump_speed = 8
                    pyxel.play(ch=0, snd=0)
            if self.jump:
                self.ninja_pos[1] -= self.jump_speed
                self.jump_speed -= 0.6
                if self.ninja_pos[1] > 112:
                    self.ninja_pos[1] = 112
                    self.jump = False
            # 按 S 键发射飞镖
            if pyxel.btnp(pyxel.KEY_S):
                self.shoot = True
                # 先将飞镖移到忍者附近位置
                self.dart_pos[0] = self.ninja_pos[0] + 16
                self.dart_pos[1] = self.ninja_pos[1] + 12
            # 如果飞镖在屏幕内，则飞镖向右移动
            if self.shoot:
                if self.dart_pos[0] > 0 and self.dart_pos[0] < 260:
                    self.dart_pos[0] += 8
                else:
                    self.shoot = False
                    self.dart_pos[1] = 200  # 将飞镖上移，这样飞镖移出屏幕后碰不到怪物

    def draw(self):
        if self.game_state == 'start':
            pyxel.text(100, 55, 'Ninja Adventure', 3)
            pyxel.text(90, 70, 'Press Space to Play!', 2)
        elif self.game_state == 'playing':
            pyxel.cls(0)
            pyxel.text(200, 15, 'Score: ' + str(self.score), 2)
            pyxel.blt(0, 136, 1, 0, 0, 250, 2, 0)               
            pyxel.blt(
                self.ninja_pos[0],
                self.ninja_pos[1],
                0, 25, 0, 22, 24, 0)  
            # 显示飞镖
            pyxel.blt(
                self.dart_pos[0],
                self.dart_pos[1],
                0, 2, 2, 12, 12, 0)                     

App()

### 2. 一波怪物正在接近
是时候在游戏中加入怪物了，由于有多个怪物，建立一个列表来存放怪物的坐标，并且用 draw() 将它们显示出来。添加检测怪物与忍者、飞镖的碰撞的功能，程序如下：

In [ ]:
import pyxel
from random import randint  # 导入randint

class App:
    def __init__(self):
        pyxel.init(250, 150, caption='Ninja Adventure')
        self.game_state = 'start'
        pyxel.load('ninja.pyxres')
        self.score = 0
        self.ninja_pos = [40, 112]
        self.jump = False
        self.jump_speed = 8
        self.dart_pos = [-20, 60]
        self.shoot = False   
        self.monster_pos = [260, 100, 280, 90, 300, 80]  # 设置怪物坐标  
        self.life = 3     
        pyxel.run(self.update, self.draw)

    def update(self):
        if self.game_state == 'start':
            if pyxel.btnp(pyxel.KEY_SPACE):  
                self.game_state = 'playing'
        elif self.game_state == 'playing':
            if pyxel.btn(pyxel.KEY_LEFT):
                self.ninja_pos[0] -= 2
            elif pyxel.btn(pyxel.KEY_RIGHT):
                self.ninja_pos[0] += 2
            if self.ninja_pos[1] > 60:          
                if pyxel.btnp(pyxel.KEY_UP):
                    self.jump = True
                    self.jump_speed = 8
                    pyxel.play(ch=0, snd=0)
            if self.jump:
                self.ninja_pos[1] -= self.jump_speed
                self.jump_speed -= 0.6
                if self.ninja_pos[1] > 112:
                    self.ninja_pos[1] = 112
                    self.jump = False
            if pyxel.btnp(pyxel.KEY_S):
                self.shoot = True
                self.dart_pos[0] = self.ninja_pos[0] + 16
                self.dart_pos[1] = self.ninja_pos[1] + 12
            if self.shoot:
                if self.dart_pos[0] > 0 and self.dart_pos[0] < 260:
                    self.dart_pos[0] += 8
                else:
                    self.shoot = False
                    self.dart_pos[1] = 200
            # 怪物移动，并检测怪物和忍者、飞镖的碰撞
            for i in range(0, 6, 2):
                self.monster_pos[i] -= 2
                if self.monster_pos[i] < -10:
                    self.monster_pos[i] = randint(270, 350)
                    self.monster_pos[i+1] = randint(70, 110)
                if abs(self.dart_pos[0] - self.monster_pos[i]) < 8 and abs(self.dart_pos[1] - self.monster_pos[i+1]) < 10:
                    self.monster_pos[i] = randint(270, 350)
                    self.monster_pos[i+1] = randint(70, 110)
                    self.score += 1 
                if abs(self.ninja_pos[0] - self.monster_pos[i]) < 20 and abs(self.ninja_pos[1] - self.monster_pos[i+1]) < 10:
                    self.monster_pos[i] = randint(270, 350)
                    self.monster_pos[i+1] = randint(70, 110)
                    self.life -= 1
            if self.life == 0:
                self.game_state = 'game over'
                pyxel.play(ch=0, snd=1)                    

    def draw(self):
        if self.game_state == 'start':
            pyxel.text(100, 55, 'Ninja Adventure', 3)
            pyxel.text(90, 70, 'Press Space to Play!', 2)
        elif self.game_state == 'playing':
            pyxel.cls(0)
            pyxel.text(200, 15, 'Score: ' + str(self.score), 2)
            pyxel.blt(0, 136, 1, 0, 0, 250, 2, 0)               
            pyxel.blt(
                self.ninja_pos[0],
                self.ninja_pos[1],
                0, 25, 0, 22, 24, 0)       
            pyxel.blt(
                self.dart_pos[0],
                self.dart_pos[1],
                0, 2, 2, 12, 12, 0)
            # 显示怪物
            for i in range(0, 6, 2):
                pyxel.blt(
                    self.monster_pos[i],
                    self.monster_pos[i+1],
                    0, 8 * i, 32, 16, 16, 0)    
App()

为了增加游戏难度，其中的一个怪物还会发射火球。继续导入火球角色，并检测火球与忍者的碰撞。

In [ ]:
import pyxel
from random import randint

class App:
    def __init__(self):
        pyxel.init(250, 150, caption='Ninja Adventure')
        self.game_state = 'start'
        pyxel.load('ninja.pyxres')
        self.score = 0
        self.ninja_pos = [40, 112]
        self.jump = False
        self.jump_speed = 8
        self.dart_pos = [-20, 60]
        self.shoot = False   
        self.monster_pos = [260, 100, 280, 90, 300, 80]  
        self.life = 3    
        self.fireball_pos = [260, 100]  # 设置火球位置
        self.fireball_shoot = False  # 火球是否发射
        pyxel.run(self.update, self.draw)

    def update(self):
        if self.game_state == 'start':
            if pyxel.btnp(pyxel.KEY_SPACE):  
                self.game_state = 'playing'
        elif self.game_state == 'playing':
            if pyxel.btn(pyxel.KEY_LEFT):
                self.ninja_pos[0] -= 2
            elif pyxel.btn(pyxel.KEY_RIGHT):
                self.ninja_pos[0] += 2
            if self.ninja_pos[1] > 60:          
                if pyxel.btnp(pyxel.KEY_UP):
                    self.jump = True
                    self.jump_speed = 8
                    pyxel.play(ch=0, snd=0)
            if self.jump:
                self.ninja_pos[1] -= self.jump_speed
                self.jump_speed -= 0.6
                if self.ninja_pos[1] > 112:
                    self.ninja_pos[1] = 112
                    self.jump = False
            if pyxel.btnp(pyxel.KEY_S):
                self.shoot = True
                self.dart_pos[0] = self.ninja_pos[0] + 16
                self.dart_pos[1] = self.ninja_pos[1] + 12
            if self.shoot:
                if self.dart_pos[0] > 0 and self.dart_pos[0] < 260:
                    self.dart_pos[0] += 8
                else:
                    self.shoot = False
                    self.dart_pos[1] = 200
            for i in range(0, 6, 2):
                self.monster_pos[i] -= 2
                if self.monster_pos[i] < -10:
                    self.monster_pos[i] = randint(270, 350)
                    self.monster_pos[i+1] = randint(70, 110)
                if abs(self.dart_pos[0] - self.monster_pos[i]) < 8 and abs(self.dart_pos[1] - self.monster_pos[i+1]) < 10:
                    self.monster_pos[i] = randint(270, 350)
                    self.monster_pos[i+1] = randint(70, 110)
                    self.score += 1 
                if abs(self.ninja_pos[0] - self.monster_pos[i]) < 20 and abs(self.ninja_pos[1] - self.monster_pos[i+1]) < 10:
                    self.monster_pos[i] = randint(270, 350)
                    self.monster_pos[i+1] = randint(70, 110)
                    self.life -= 1
            if self.life == 0:
                self.game_state = 'game over'
                pyxel.play(ch=0, snd=1)                    
            # 怪物移到某个位置时发射火球
            if self.monster_pos[0] >= 246 and self.monster_pos[0] < 248:
                self.fireball_shoot = True
                self.fireball_pos[0] = self.monster_pos[0] + 8
                self.fireball_pos[1] = self.monster_pos[1] + 8
            if self.fireball_shoot:
                self.fireball_pos[0] -= 3
                if self.fireball_pos[0] < -10:
                    self.fireball_shoot = False
            # 检测火球与忍者的碰撞
            if abs(self.ninja_pos[0] - self.fireball_pos[0]) < 20 and abs(self.ninja_pos[1] - self.fireball_pos[1]) < 6:
                self.fireball_pos = [260, 100]
                self.life -= 1

    def draw(self):
        if self.game_state == 'start':
            pyxel.text(100, 55, 'Ninja Adventure', 3)
            pyxel.text(90, 70, 'Press Space to Play!', 2)
        elif self.game_state == 'playing':
            pyxel.cls(0)
            pyxel.text(200, 15, 'Score: ' + str(self.score), 2)
            pyxel.blt(0, 136, 1, 0, 0, 250, 2, 0)               
            pyxel.blt(
                self.ninja_pos[0],
                self.ninja_pos[1],
                0, 25, 0, 22, 24, 0)       
            pyxel.blt(
                self.dart_pos[0],
                self.dart_pos[1],
                0, 2, 2, 12, 12, 0)                     
            for i in range(0, 6, 2):
                pyxel.blt(
                    self.monster_pos[i],
                    self.monster_pos[i+1],
                    0, 8 * i, 32, 16, 16, 0)  
            # 显示火球
            pyxel.blt(
                self.fireball_pos[0],
                self.fireball_pos[1],
                0, 4, 53, 8, 8, 0)  
                                     
App()

为了增加续航，屏幕中不时会有心形落下，补充生命值。在游戏屏幕左上角用心形表示生命值，如下图所示：

<img src='image/ch16.5.png' width=800 align=left>

相应代码如下：

In [ ]:
import pyxel
from random import randint

class App:
    def __init__(self):
        pyxel.init(250, 150, caption='Ninja Adventure')
        self.game_state = 'start'
        pyxel.load('ninja.pyxres')
        self.score = 0
        self.ninja_pos = [40, 112]
        self.jump = False
        self.jump_speed = 8
        self.dart_pos = [-20, 60]
        self.shoot = False   
        self.monster_pos = [260, 100, 280, 90, 300, 80]  
        self.life = 3    
        self.fireball_pos = [260, 100]
        self.fireball_shoot = False 
        self.heart_pos = [120, -500]  # 设置下落心形的位置
        pyxel.run(self.update, self.draw)

    def update(self):
        if self.game_state == 'start':
            if pyxel.btnp(pyxel.KEY_SPACE):  
                self.game_state = 'playing'
        elif self.game_state == 'playing':
            if pyxel.btn(pyxel.KEY_LEFT):
                self.ninja_pos[0] -= 2
            elif pyxel.btn(pyxel.KEY_RIGHT):
                self.ninja_pos[0] += 2
            if self.ninja_pos[1] > 60:          
                if pyxel.btnp(pyxel.KEY_UP):
                    self.jump = True
                    self.jump_speed = 8
                    pyxel.play(ch=0, snd=0)
            if self.jump:
                self.ninja_pos[1] -= self.jump_speed
                self.jump_speed -= 0.6
                if self.ninja_pos[1] > 112:
                    self.ninja_pos[1] = 112
                    self.jump = False
            if pyxel.btnp(pyxel.KEY_S):
                self.shoot = True
                self.dart_pos[0] = self.ninja_pos[0] + 16
                self.dart_pos[1] = self.ninja_pos[1] + 12
            if self.shoot:
                if self.dart_pos[0] > 0 and self.dart_pos[0] < 260:
                    self.dart_pos[0] += 8
                else:
                    self.shoot = False
                    self.dart_pos[1] = 200
            for i in range(0, 6, 2):
                self.monster_pos[i] -= 2
                if self.monster_pos[i] < -10:
                    self.monster_pos[i] = randint(270, 350)
                    self.monster_pos[i+1] = randint(70, 110)
                if abs(self.dart_pos[0] - self.monster_pos[i]) < 8 and abs(self.dart_pos[1] - self.monster_pos[i+1]) < 10:
                    self.monster_pos[i] = randint(270, 350)
                    self.monster_pos[i+1] = randint(70, 110)
                    self.score += 1 
                if abs(self.ninja_pos[0] - self.monster_pos[i]) < 20 and abs(self.ninja_pos[1] - self.monster_pos[i+1]) < 10:
                    self.monster_pos[i] = randint(270, 350)
                    self.monster_pos[i+1] = randint(70, 110)
                    self.life -= 1
            if self.life == 0:
                self.game_state = 'game over'
                pyxel.play(ch=0, snd=1)                    

            if self.monster_pos[0] >= 246 and self.monster_pos[0] < 248:
                self.fireball_shoot = True
                self.fireball_pos[0] = self.monster_pos[0] + 8
                self.fireball_pos[1] = self.monster_pos[1] + 8
            if self.fireball_shoot:
                self.fireball_pos[0] -= 3
                if self.fireball_pos[0] < -10:
                    self.fireball_shoot = False
            if abs(self.ninja_pos[0] - self.fireball_pos[0]) < 20 and abs(self.ninja_pos[1] - self.fireball_pos[1]) < 6:
                self.fireball_pos = [260, 100]
                self.life -= 1
            # 心形下落，检测心形和忍者的碰撞
            self.heart_pos[1] += 2
            if abs(self.ninja_pos[0] - self.heart_pos[0]) < 12 and abs(self.ninja_pos[1] - self.heart_pos[1]) < 12:
                if self.life < 3:
                    self.life += 1
                self.heart_pos[0] = randint(60, 160)
                self.heart_pos[1] = randint(-1000, -500)
            if self.heart_pos[1] > 160:
                self.heart_pos[0] = randint(60, 160)
                self.heart_pos[1] = randint(-1000, -500)

                
    def draw(self):
        if self.game_state == 'start':
            pyxel.text(100, 55, 'Ninja Adventure', 3)
            pyxel.text(90, 70, 'Press Space to Play!', 2)
        elif self.game_state == 'playing':
            pyxel.cls(0)
            pyxel.text(200, 15, 'Score: ' + str(self.score), 2)
            pyxel.blt(0, 136, 1, 0, 0, 250, 2, 0)               
            pyxel.blt(
                self.ninja_pos[0],
                self.ninja_pos[1],
                0, 25, 0, 22, 24, 0)       
            pyxel.blt(
                self.dart_pos[0],
                self.dart_pos[1],
                0, 2, 2, 12, 12, 0)                     
            for i in range(0, 6, 2):
                pyxel.blt(
                    self.monster_pos[i],
                    self.monster_pos[i+1],
                    0, 8 * i, 32, 16, 16, 0)   
            pyxel.blt(
                self.fireball_pos[0],
                self.fireball_pos[1],
                0, 4, 53, 8, 8, 0) 
            # 显示下落的心形和心形生命值
            pyxel.blt(10, 6, 0, 0, 68, 16*self.life, 12, 0) 
            pyxel.blt(
                self.heart_pos[0],
                self.heart_pos[1],
                0, 0, 68, 16, 12, 0)                 

App()

## 游戏升级
忍者想憋一个“瞬移”的大招，问题不大，瞬移就是瞬间改变坐标，在程序中几行代码就能搞定。不过，既然这个招式是憋出来的，自然有个蓄力的过程，这里引入蓝量的概念，只有蓝量是满的情况下，才能瞬移。瞬移会耗空蓝量，不过蓝量随着时间会慢慢回复。

<img src='image/ch16.6.png' width=190 align=left>

In [ ]:
import pyxel
from random import randint

class App:
    def __init__(self):
        pyxel.init(250, 150, caption='Ninja Adventure')
        self.game_state = 'start'
        pyxel.load('ninja.pyxres')
        self.score = 0
        self.ninja_pos = [40, 112]
        self.jump = False
        self.jump_speed = 8
        self.dart_pos = [-20, 60]
        self.shoot = False   
        self.monster_pos = [260, 100, 280, 90, 300, 80]  
        self.life = 3    
        self.fireball_pos = [260, 100]
        self.fireball_shoot = False 
        self.heart_pos = [120, -500]
        self.mp = 3  # 设置蓝量值
        self.count = 0  # 设置蓝量回复计数
        pyxel.run(self.update, self.draw)

    def update(self):
        if self.game_state == 'start':
            if pyxel.btnp(pyxel.KEY_SPACE):  
                self.game_state = 'playing'
        elif self.game_state == 'playing':
            if pyxel.btn(pyxel.KEY_LEFT):
                self.ninja_pos[0] -= 2
            elif pyxel.btn(pyxel.KEY_RIGHT):
                self.ninja_pos[0] += 2
            if self.ninja_pos[1] > 60:          
                if pyxel.btnp(pyxel.KEY_UP):
                    self.jump = True
                    self.jump_speed = 8
                    pyxel.play(ch=0, snd=0)
            if self.jump:
                self.ninja_pos[1] -= self.jump_speed
                self.jump_speed -= 0.6
                if self.ninja_pos[1] > 112:
                    self.ninja_pos[1] = 112
                    self.jump = False
            if pyxel.btnp(pyxel.KEY_S):
                self.shoot = True
                self.dart_pos[0] = self.ninja_pos[0] + 16
                self.dart_pos[1] = self.ninja_pos[1] + 12
            if self.shoot:
                if self.dart_pos[0] > 0 and self.dart_pos[0] < 260:
                    self.dart_pos[0] += 8
                else:
                    self.shoot = False
                    self.dart_pos[1] = 200
            for i in range(0, 6, 2):
                self.monster_pos[i] -= 2
                if self.monster_pos[i] < -10:
                    self.monster_pos[i] = randint(270, 350)
                    self.monster_pos[i+1] = randint(70, 110)
                if abs(self.dart_pos[0] - self.monster_pos[i]) < 8 and abs(self.dart_pos[1] - self.monster_pos[i+1]) < 10:
                    self.monster_pos[i] = randint(270, 350)
                    self.monster_pos[i+1] = randint(70, 110)
                    self.score += 1 
                if abs(self.ninja_pos[0] - self.monster_pos[i]) < 20 and abs(self.ninja_pos[1] - self.monster_pos[i+1]) < 10:
                    self.monster_pos[i] = randint(270, 350)
                    self.monster_pos[i+1] = randint(70, 110)
                    self.life -= 1
            if self.life == 0:
                self.game_state = 'game over'
                pyxel.play(ch=0, snd=1)                    

            if self.monster_pos[0] >= 246 and self.monster_pos[0] < 248:
                self.fireball_shoot = True
                self.fireball_pos[0] = self.monster_pos[0] + 8
                self.fireball_pos[1] = self.monster_pos[1] + 8
            if self.fireball_shoot:
                self.fireball_pos[0] -= 3
                if self.fireball_pos[0] < -10:
                    self.fireball_shoot = False
            if abs(self.ninja_pos[0] - self.fireball_pos[0]) < 20 and abs(self.ninja_pos[1] - self.fireball_pos[1]) < 6:
                self.fireball_pos = [260, 100]
                self.life -= 1

            self.heart_pos[1] += 2
            if abs(self.ninja_pos[0] - self.heart_pos[0]) < 12 and abs(self.ninja_pos[1] - self.heart_pos[1]) < 12:
                if self.life < 3:
                    self.life += 1
                self.heart_pos[0] = randint(60, 160)
                self.heart_pos[1] = randint(-1000, -500)
            if self.heart_pos[1] > 160:
                self.heart_pos[0] = randint(60, 160)
                self.heart_pos[1] = randint(-1000, -500)
            # 按 D 键瞬移
            if pyxel.btnp(pyxel.KEY_D):
                if self.mp == 3:
                    self.mp = 0
                    self.ninja_pos[0] = min(self.ninja_pos[0] + 100, 240)
                    self.count = 1
            # 回复蓝量值
            if self.count:
                self.count += 1
                self.mp = int(self.count // 200)
                if self.mp == 3:
                    self.count = 0            


    def draw(self):
        if self.game_state == 'start':
            pyxel.text(100, 55, 'Ninja Adventure', 3)
            pyxel.text(90, 70, 'Press Space to Play!', 2)
        elif self.game_state == 'playing':
            pyxel.cls(0)
            pyxel.text(200, 15, 'Score: ' + str(self.score), 2)
            pyxel.blt(0, 136, 1, 0, 0, 250, 2, 0)               
            pyxel.blt(
                self.ninja_pos[0],
                self.ninja_pos[1],
                0, 25, 0, 22, 24, 0)       
            pyxel.blt(
                self.dart_pos[0],
                self.dart_pos[1],
                0, 2, 2, 12, 12, 0)                     
            for i in range(0, 6, 2):
                pyxel.blt(
                    self.monster_pos[i],
                    self.monster_pos[i+1],
                    0, 8 * i, 32, 16, 16, 0)   
            pyxel.blt(
                self.fireball_pos[0],
                self.fireball_pos[1],
                0, 4, 53, 8, 8, 0) 
            pyxel.blt(10, 6, 0, 0, 68, 16*self.life, 12, 0) 
            pyxel.blt(
                self.heart_pos[0],
                self.heart_pos[1],
                0, 0, 68, 16, 12, 0)                 
            pyxel.blt(10, 22, 0, 0, 85, 16*self.mp, 12, 0)  # 显示蓝量值

App()

再加上游戏结束后，按下按键重新开始的功能，这样一个游戏就比较完整了。

In [ ]:
import pyxel
from random import randint

class App:
    def __init__(self):
        pyxel.init(250, 150, caption='Ninja Adventure')
        self.game_state = 'start'
        pyxel.load('ninja.pyxres')
        self.score = 0
        self.ninja_pos = [40, 112]
        self.jump = False
        self.jump_speed = 8
        self.dart_pos = [-20, 60]
        self.shoot = False   
        self.monster_pos = [260, 100, 280, 90, 300, 80]  
        self.life = 3    
        self.fireball_pos = [260, 100]
        self.fireball_shoot = False 
        self.heart_pos = [120, -500]
        self.mp = 3
        self.count = 0  
        pyxel.run(self.update, self.draw)

    def update(self):
        if self.game_state == 'start':
            if pyxel.btnp(pyxel.KEY_SPACE):  
                self.game_state = 'playing'
        elif self.game_state == 'playing':
            if pyxel.btn(pyxel.KEY_LEFT):
                self.ninja_pos[0] -= 2
            elif pyxel.btn(pyxel.KEY_RIGHT):
                self.ninja_pos[0] += 2
            if self.ninja_pos[1] > 60:          
                if pyxel.btnp(pyxel.KEY_UP):
                    self.jump = True
                    self.jump_speed = 8
                    pyxel.play(ch=0, snd=0)
            if self.jump:
                self.ninja_pos[1] -= self.jump_speed
                self.jump_speed -= 0.6
                if self.ninja_pos[1] > 112:
                    self.ninja_pos[1] = 112
                    self.jump = False
            if pyxel.btnp(pyxel.KEY_S):
                self.shoot = True
                self.dart_pos[0] = self.ninja_pos[0] + 16
                self.dart_pos[1] = self.ninja_pos[1] + 12
            if self.shoot:
                if self.dart_pos[0] > 0 and self.dart_pos[0] < 260:
                    self.dart_pos[0] += 8
                else:
                    self.shoot = False
                    self.dart_pos[1] = 200
            for i in range(0, 6, 2):
                self.monster_pos[i] -= 2
                if self.monster_pos[i] < -10:
                    self.monster_pos[i] = randint(270, 350)
                    self.monster_pos[i+1] = randint(70, 110)
                if abs(self.dart_pos[0] - self.monster_pos[i]) < 8 and abs(self.dart_pos[1] - self.monster_pos[i+1]) < 10:
                    self.monster_pos[i] = randint(270, 350)
                    self.monster_pos[i+1] = randint(70, 110)
                    self.score += 1 
                if abs(self.ninja_pos[0] - self.monster_pos[i]) < 20 and abs(self.ninja_pos[1] - self.monster_pos[i+1]) < 10:
                    self.monster_pos[i] = randint(270, 350)
                    self.monster_pos[i+1] = randint(70, 110)
                    self.life -= 1
            if self.life == 0:
                self.game_state = 'game over'
                pyxel.play(ch=0, snd=1)                    

            if self.monster_pos[0] >= 246 and self.monster_pos[0] < 248:
                self.fireball_shoot = True
                self.fireball_pos[0] = self.monster_pos[0] + 8
                self.fireball_pos[1] = self.monster_pos[1] + 8
            if self.fireball_shoot:
                self.fireball_pos[0] -= 3
                if self.fireball_pos[0] < -10:
                    self.fireball_shoot = False
            if abs(self.ninja_pos[0] - self.fireball_pos[0]) < 20 and abs(self.ninja_pos[1] - self.fireball_pos[1]) < 6:
                self.fireball_pos = [260, 100]
                self.life -= 1

            self.heart_pos[1] += 2
            if abs(self.ninja_pos[0] - self.heart_pos[0]) < 12 and abs(self.ninja_pos[1] - self.heart_pos[1]) < 12:
                if self.life < 3:
                    self.life += 1
                self.heart_pos[0] = randint(60, 160)
                self.heart_pos[1] = randint(-1000, -500)
            if self.heart_pos[1] > 160:
                self.heart_pos[0] = randint(60, 160)
                self.heart_pos[1] = randint(-1000, -500)

            if pyxel.btnp(pyxel.KEY_D):
                if self.mp == 3:
                    self.mp = 0
                    self.ninja_pos[0] = min(self.ninja_pos[0] + 100, 240)
                    self.count = 1
            if self.count:
                self.count += 1
                self.mp = int(self.count // 200)
                if self.mp == 3:
                    self.count = 0            


    def draw(self):
        if self.game_state == 'start':
            pyxel.text(100, 55, 'Ninja Adventure', 3)
            pyxel.text(90, 70, 'Press Space to Play!', 2)
        elif self.game_state == 'playing':
            pyxel.cls(0)
            pyxel.text(200, 15, 'Score: ' + str(self.score), 2)
            pyxel.blt(0, 136, 1, 0, 0, 250, 2, 0)               
            pyxel.blt(
                self.ninja_pos[0],
                self.ninja_pos[1],
                0, 25, 0, 22, 24, 0)       
            pyxel.blt(
                self.dart_pos[0],
                self.dart_pos[1],
                0, 2, 2, 12, 12, 0)                     
            for i in range(0, 6, 2):
                pyxel.blt(
                    self.monster_pos[i],
                    self.monster_pos[i+1],
                    0, 8 * i, 32, 16, 16, 0)   
            pyxel.blt(
                self.fireball_pos[0],
                self.fireball_pos[1],
                0, 4, 53, 8, 8, 0) 
            pyxel.blt(10, 6, 0, 0, 68, 16*self.life, 12, 0) 
            pyxel.blt(
                self.heart_pos[0],
                self.heart_pos[1],
                0, 0, 68, 16, 12, 0)                 
            pyxel.blt(10, 22, 0, 0, 85, 16*self.mp, 12, 0)
        else:
            pyxel.text(110, 55, 'Game Over', 8)
            # 按下空格键重新开始游戏，重置游戏状态
            pyxel.text(85, 70, 'Press Space to Restart!', 2)
            if pyxel.btnp(pyxel.KEY_SPACE):
                self.game_state = 'playing'
                self.score = 0
                self.life = 3
                self.monster_pos = [260, 100, 280, 90, 300, 80] 
                self.heart_pos = [120, -500]
                self.count = 0
                self.mp = 3

App()

游戏中出现了多个怪物，如果定义一个怪物类 Monster，程序应该如何编写？还想添加什么功能？杀死 20 只怪物的时候出现大 Boss？继续改进这个游戏吧！

In [1]:
from IPython.display import HTML

HTML('<iframe frameborder="no" border="0" marginwidth="0" marginheight="0" width=330 height=86 src="//music.163.com/outchain/player?type=2&id=27969765&auto=0&height=66"></iframe>')